## Risk parity portfolio

In [17]:
from alpaca.data import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from load_api_keys import load_alpaca_paper_trading_keys
import datetime

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass, OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest
from clients import AlpacaCryptoTradingClient

api_keys = load_alpaca_paper_trading_keys()

crypto_client = CryptoHistoricalDataClient()

In [2]:
trading_client = TradingClient(api_key=api_keys.api_key, secret_key=api_keys.secret_key)
search_params = GetAssetsRequest(asset_class=AssetClass.CRYPTO)
all_cryptos = trading_client.get_all_assets(search_params)

We are only interested in markets where there is US Dollar against the coin. Let's see those

In [3]:
crypto_usd_markets = []
for asset in all_cryptos: 
    if 'US Dollar' in asset.name: 
        crypto_usd_markets.append(asset)

In [4]:
crypto_usd_markets_symbols = [i.symbol for i in crypto_usd_markets if i.symbol not in ['DAI/USD', 'PAXG/USD', 'USDT/USD', 'TRX/USD']]

In [5]:
# Then we get all the daily data for the past year for each of these assets
crypto_bars_request = CryptoBarsRequest(
    symbol_or_symbols=crypto_usd_markets_symbols, 
    timeframe=TimeFrame.Day, 
    start = datetime.datetime(2022, 1, 1)
)

crypto_bars = crypto_client.get_crypto_bars(crypto_bars_request)

How are we going to come up with a volatility score for each here? 

In [6]:
crypto_bars_df = crypto_bars.df

In [7]:
product = crypto_usd_markets_symbols[0]
product_df = crypto_bars_df.loc[(product, slice(None))]

In [8]:
def calculate_volatility(df): 

    return df['close'].pct_change().dropna().std()

In [9]:
realized_volatilities = crypto_bars_df.reset_index().groupby('symbol').apply(calculate_volatility)

In [10]:
returns = crypto_bars_df.reset_index().pivot(index = ['timestamp'], columns = ['symbol'], values = 'close').dropna().pct_change().dropna()

In [11]:
covariance_matrix = returns.cov()
total_variation = covariance_matrix.sum().sum()
total_weight_scores = total_variation/covariance_matrix.sum(axis = 0)

In [12]:
total_weight_scores = total_variation/covariance_matrix.sum(axis = 0)

In [13]:
asset_weights = total_weight_scores/total_weight_scores.sum()

In [14]:
dollars = 10_000
asset_amounts_in_dollars = asset_weights * dollars

In [16]:
asset_amounts_in_dollars['AAVE/USD']

394.14248944745884

In [27]:
# Now that we have this portfolio, we can make all the orders to do this
market_orders = []
for i in range(asset_amounts_in_dollars.shape[0]):
    market = asset_amounts_in_dollars.index[i]

    market_order_data = MarketOrderRequest(
        symbol = market, 
        notional = asset_amounts_in_dollars[market], 
        side = OrderSide.BUY, 
        time_in_force = TimeInForce.GTC
    )

    market_orders.append(market_order_data)

In [28]:
completed_orders = {}
for market_order in market_orders: 
    order = trading_client.submit_order(order_data = market_order)
    completed_orders[order.symbol] = order

In [29]:
completed_orders

{'AAVE/USD': {   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('8588af33-3a8e-4a09-bb6c-ba4226dc7aff'),
     'canceled_at': None,
     'client_order_id': '402fa188-05c2-4c1f-9e3b-fe6cbe6611bb',
     'created_at': datetime.datetime(2022, 10, 17, 1, 49, 44, 808582, tzinfo=datetime.timezone.utc),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('e62ef293-6cd2-407c-bbbe-7fc76723bcdb'),
     'legs': None,
     'limit_price': None,
     'notional': '394.142489447',
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.MARKET: 'market'>,
     'qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
     'stop_price': None,
     'submitted_at': datetime.datetime(2022, 10, 17, 1, 49,